# 3. Sp_normalization.Rmd
    + count S.pombe reads
    + derive scaling factors using DESeq2
    + apply to scaling of S. cerevisiae tracks
    + sanity check that scaling worked OK

Prep S. pombe annotations used

In [1]:

### get S.pombe annotations for gene TSS to TES + 300bp
cd ~/annotations/Spombe_ENSEMBLE/

#extract exons and convert to 0-based
cat Sp_genes.gtf | awk '{if($3=="exon"){sub("\"","",$12);sub("\";","",$12);print $1"\t"$4-1"\t"$5"\t"$12"\t"$3"\t"$7}}' > Sp_exons.bed

#combine exons of each annotation and extend end by 300bp after TES
awk '$6=="+"' Sp_exons.bed | sort -k4,4 -k2,2n | \
awk '
{
  if($4==i){
    e=$3+300
  }else{
    print c"\t"s"\t"e"\t"i"\t"t"\t"str; 
    c=$1;s=$2;e=$3+300;i=$4;t=$5;str=$6
  }
}END{
  print c"\t"s"\t"e"\t"i"\t"t"\t"str
}' | sed 1d | sort -k1,1 -k2,2n > Sp_exons_ends_p300_plus.bed
awk '$6=="-"' Sp_exons.bed | sort -k4,4 -k2,2n | awk '{if($4==i){e=$3}else{print c"\t"s"\t"e"\t"i"\t"t"\t"str; c=$1;s=$2-300;if(s < 0){s=0};e=$3;i=$4;t=$5;str=$6}}END{print c"\t"s"\t"e"\t"i"\t"t"\t"str}' | sed 1d | sort -k1,1 -k2,2n > Sp_exons_ends_p300_minus.bed

## improved version removing overlaps between neighbors on same strand
awk '{OFS="\t"}{if(chr == $1 && $2 < end){if(end<$3){$2=end;$5="trimmed_gene_plus300bp";print $0}}else{$5="gene_plus300bp"; print $0};chr=$1;end=$3}' Sp_exons_ends_p300_plus.bed > Sp_gene_ends_p300_nonoverlapping_plus.bed 
awk '{OFS="\t"}{if(chr == $1 && $2 < end){if(end<$3){$2=end;$5="trimmed_gene_plus300bp";print $0}}else{$5="gene_plus300bp"; print $0};chr=$1;end=$3}' Sp_exons_ends_p300_minus.bed > Sp_gene_ends_p300_nonoverlapping_minus.bed 


Get counts for each Spombe transcript 10min data

In [4]:
cd ~/output/STAR_map/

mkdir -p raw_bedgraph/Spombe_bedgraphs

for f in *.bedgraph
  do 
    grep -v ^chr $f > ~/output/STAR_map/${f/.bedgraph/_Sp.bedgraph}
  done
  
mkdir -p Spombe_per_gene_counts

plus_bed="~/annotations/Spombe_ENSEMBLE/Sp_gene_ends_p300_nonoverlapping_plus.bed"
minus_bed="~/annotations/Spombe_ENSEMBLE/Sp_gene_ends_p300_nonoverlapping_minus.bed"

cd ~/output/STAR_map/raw_bedgraph/Spombe_bedgraphs

mkdir -p Spombe_per_gene_counts

for b in *_plus_Sp.bedgraph
do
  echo "counting plus strand file ${b}"
  bash ~/software/bed_count.sh $plus_bed $b 4 Spombe_per_gene_counts/${b/_plus_Sp.bedgraph/_Sp_plus.raw_counts}
  echo "   minus strand"
  bash ~/software/bed_count.sh $minus_bed ${b/plus/minus} 4 Spombe_per_gene_counts/${b/_plus_Sp.bedgraph/_Sp_minus.raw_counts}
  cat Spombe_per_gene_counts/${b/_plus_Sp.bedgraph/_Sp_plus.raw_counts} Spombe_per_gene_counts/${b/_plus_Sp.bedgraph/_Sp_minus.raw_counts} | sort -k1,1 -k2,2n > Spombe_per_gene_counts/${b/_plus_Sp.bedgraph/_Sp.raw_counts}
  rm Spombe_per_gene_counts/${b/_plus_Sp.bedgraph/_Sp_plus.raw_counts}
  rm Spombe_per_gene_counts/${b/_plus_Sp.bedgraph/_Sp_minus.raw_counts}
done

counting plus strand file *_plus_Sp.bedgraph
/home/ubuntu/software/bed_count.sh: line 24: syntax error near unexpected token `newline'
/home/ubuntu/software/bed_count.sh: line 24: `}'  | sort -k1,1 -k2,2n | bedtools map -a  -b - -c  -o sum > '
   minus strand
/home/ubuntu/software/bed_count.sh: line 24: syntax error near unexpected token `newline'
/home/ubuntu/software/bed_count.sh: line 24: `}'  | sort -k1,1 -k2,2n | bedtools map -a  -b - -c  -o sum > '
cat: 'Spombe_per_gene_counts/*_Sp_plus.raw_counts': No such file or directory
cat: 'Spombe_per_gene_counts/*_Sp_minus.raw_counts': No such file or directory
rm: cannot remove 'Spombe_per_gene_counts/*_Sp_plus.raw_counts': No such file or directory
rm: cannot remove 'Spombe_per_gene_counts/*_Sp_minus.raw_counts': No such file or directory


: 1